In [8]:
import requests
import json
from solders.keypair import Keypair
from solana.rpc.api import Client



# Solana API endpoint
url = "https://solana-mainnet.g.alchemy.com/v2/KgpuMjqocuyieV47x-HcBYGyi8LrIFb8"

# Request payload
payload = {
    "jsonrpc": "2.0",
    "id": 1,
    "method": "getAccountInfo",
    "params": [
        "8h3eQCUq4sfKraaSGgoQM2wt1kjCYq6BoaAkCrPt3waa",
        {
            "encoding": "base58"
        }
    ]
}

# Headers
headers = {
    "Content-Type": "application/json"
}

# Make the POST request
response = requests.post(url, headers=headers, data=json.dumps(payload))

# Check if the request was successful
if response.status_code == 200:
    account_info = response.json()
    print(json.dumps(account_info, indent=4))  # Pretty-print the JSON response
else:
    print(f"Error: {response.status_code}")
    print(response.text)

{
    "jsonrpc": "2.0",
    "result": {
        "context": {
            "apiVersion": "1.18.15",
            "slot": 275293180
        },
        "value": {
            "data": [
                "",
                "base58"
            ],
            "executable": false,
            "lamports": 1519362,
            "owner": "11111111111111111111111111111111",
            "rentEpoch": 18446744073709551615,
            "space": 0
        }
    },
    "id": 1
}


In [9]:


def get_quote(src_chain_id, src_chain_token_in, src_chain_token_in_amount, dst_chain_id, dst_chain_token_out, affiliate_fee_percent, prepend_operating_expenses):
    url = "https://api.dln.trade/v1.0/dln/order/quote"
    params = {
        "srcChainId": src_chain_id,
        "srcChainTokenIn": src_chain_token_in,
        "srcChainTokenInAmount": src_chain_token_in_amount,
        "dstChainId": dst_chain_id,
        "dstChainTokenOut": dst_chain_token_out,
        "dstChainTokenOutAmount": "auto",
        "affiliateFeePercent": affiliate_fee_percent,
        "prependOperatingExpenses": prepend_operating_expenses
    }
    
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        response.raise_for_status()

# Example usage
src_chain_id = 7565164  # Arbitrum
src_chain_token_in = "EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v"  # USDC on Arbitrum
src_chain_token_in_amount = 500000  # 50 USDC in smallest unit (assuming 6 decimals)
dst_chain_id = 42161  # Solana
dst_chain_token_out = "0xaf88d065e77c8cC2239327C5EDb3A432268e5831"  # USDC on Solana
affiliate_fee_percent = 0.0916
prepend_operating_expenses = True

try:
    quote = get_quote(src_chain_id, src_chain_token_in, src_chain_token_in_amount, dst_chain_id, dst_chain_token_out, affiliate_fee_percent, prepend_operating_expenses)
    print(json.dumps(quote, indent=4))  # Pretty-print the JSON response
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")



{
    "estimation": {
        "srcChainTokenIn": {
            "name": "USD Coin",
            "symbol": "USDC",
            "chainId": 7565164,
            "address": "EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v",
            "decimals": 6,
            "amount": "1656560",
            "approximateOperatingExpense": "1154596",
            "mutatedWithOperatingExpense": true
        },
        "dstChainTokenOut": {
            "address": "0xaf88d065e77c8cc2239327c5edb3a432268e5831",
            "chainId": 42161,
            "decimals": 6,
            "name": "USD Coin",
            "symbol": "USDC",
            "amount": "499151",
            "recommendedAmount": "499151",
            "maxTheoreticalAmount": "499151"
        },
        "costsDetails": [
            {
                "chain": "7565164",
                "tokenIn": "EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v",
                "tokenOut": "EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v",
                "amountIn": "165656

In [9]:
quotetx = quote.get('tx', {})
allow_target = quotetx.get('allowanceTarget')
allow_value = quotetx.get('allowanceValue')
dstchaintokenout1 = quotetx.get('dstChainTokenOut', {})
dstchaintokenout = dstchaintokenout1.get('recommendedAmount')

In [10]:


def create_tx(src_chain_id, src_chain_token_in, src_chain_token_in_amount, dst_chain_id, dst_chain_token_out, dst_chain_token_out_amount, 
            dst_chain_token_out_recipient, src_chain_order_authority_address, affiliate_fee_percent, affiliate_fee_recipient,
            dst_chain_order_authority_address, prepend_operating_expenses, referral_code):
    url = "https://api.dln.trade/v1.0/dln/order/create-tx"
    params = {
        "srcChainId": src_chain_id,
        "srcChainTokenIn": src_chain_token_in,
        "srcChainTokenInAmount": src_chain_token_in_amount,
        "dstChainId": dst_chain_id,
        "dstChainTokenOut": dst_chain_token_out,
        "dstChainTokenOutAmount": dst_chain_token_out_amount,
        "dstChainTokenOutRecipient": dst_chain_token_out_recipient,
        "srcChainOrderAuthorityAddress": src_chain_order_authority_address,
        "affiliateFeePercent": affiliate_fee_percent,
        "affiliateFeeRecipient": affiliate_fee_recipient,
        "dstChainOrderAuthorityAddress": dst_chain_order_authority_address,
        "prependOperatingExpenses": prepend_operating_expenses,
        "referralCode": referral_code
    }
    
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        response.raise_for_status()
# Example usage
src_chain_id = 7565164  # solana
src_chain_token_in = "EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v"  # USDC on solana
src_chain_token_in_amount = 500000  # 50 USDC in smallest unit (assuming 6 decimals)
dst_chain_id = 42161  # Solana
dst_chain_token_out = "0xaf88d065e77c8cC2239327C5EDb3A432268e5831"  # USDC on Solana
dst_chain_token_out_amount = "auto"
dst_chain_token_out_recipient = "0xef772390c8aDD25302be26Fb92edb7F5dE214480"
src_chain_order_authority_address = "8h3eQCUq4sfKraaSGgoQM2wt1kjCYq6BoaAkCrPt3waa"
affiliate_fee_percent = 0.1
affiliate_fee_recipient = "0x09fe3a3EA67eD2fEceFa9Af22380e53f0F580614"
dst_chain_order_authority_address = "0xef772390c8aDD25302be26Fb92edb7F5dE214480"
prepend_operating_expenses = False
referral_code = "15434"

try:
    tx = create_tx(src_chain_id, src_chain_token_in, src_chain_token_in_amount, dst_chain_id, dst_chain_token_out, dst_chain_token_out_amount, dst_chain_token_out_recipient, src_chain_order_authority_address, affiliate_fee_percent, affiliate_fee_recipient, dst_chain_order_authority_address, prepend_operating_expenses, referral_code)
    print(json.dumps(tx, indent=4))  # Pretty-print the JSON response
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

An error occurred: 400 Client Error: Bad Request for url: https://api.dln.trade/v1.0/dln/order/create-tx?srcChainId=7565164&srcChainTokenIn=EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v&srcChainTokenInAmount=500000&dstChainId=42161&dstChainTokenOut=0xaf88d065e77c8cC2239327C5EDb3A432268e5831&dstChainTokenOutAmount=auto&dstChainTokenOutRecipient=0xef772390c8aDD25302be26Fb92edb7F5dE214480&srcChainOrderAuthorityAddress=8h3eQCUq4sfKraaSGgoQM2wt1kjCYq6BoaAkCrPt3waa&affiliateFeePercent=0.1&affiliateFeeRecipient=0x09fe3a3EA67eD2fEceFa9Af22380e53f0F580614&dstChainOrderAuthorityAddress=0xef772390c8aDD25302be26Fb92edb7F5dE214480&prependOperatingExpenses=False&referralCode=15434


In [52]:
import requests
import json
from web3 import Web3
from eth_account import Account
import os

# Solana setup
solana_client = Client("https://solana-mainnet.g.alchemy.com/v2/KgpuMjqocuyieV47x-HcBYGyi8LrIFb8")
private_key = os.getenv('PRIVATE_KEY')



In [8]:
solana_client

In [2]:
from web3 import Web3, AsyncWeb3
from solders.message import Message
from solders.keypair import Keypair
from solders.instruction import Instruction
from solders.hash import Hash
from solders.transaction import Transaction
from solders.pubkey import Pubkey
import os
from dotenv import load_dotenv
import base58

# Load environment variables from .env file
load_dotenv()

# Get the private key from the environment variable
pk_base58 = os.getenv('PRIVATE_KEY')

if pk_base58 is None:
    raise ValueError("The private key must be set in the .env file.")

# Decode the base58 private key to bytes
pk_bytes = base58.b58decode(pk_base58)

# Ensure the private key is 32 bytes long
if len(pk_bytes) != 32:
    raise ValueError("The private key must be exactly 32 bytes long.")

# Convert the bytes to a hexadecimal string
pk_hex = pk_bytes.hex()

w3 = Web3(Web3.HTTPProvider("https://solana-mainnet.g.alchemy.com/v2/KgpuMjqocuyieV47x-HcBYGyi8LrIFb8"))

# Create an account object from the private key
account = w3.eth.account.from_key(pk_hex)

ValueError: The private key must be exactly 32 bytes long.

In [25]:
import solana
from solana.rpc.api import Client

solclient = Client('https://solana-mainnet.g.alchemy.com/v2/KgpuMjqocuyieV47x-HcBYGyi8LrIFb8')



In [6]:
from solders.keypair import Keypair
from solders.pubkey import Pubkey
from solders.instruction import Instruction, AccountMeta





keypair = Keypair.from_base58_string(os.getenv('PRIVATE_KEY'))
# Create the Pubkey objects
program_id = Pubkey.from_string("ATokenGPvbdGVxr1b2hvZbsiqW5xWH25efTNsLJA8knL")
auth_pubkey = Pubkey.from_string("2iBUASRfDHgEkuZ91Lvos5NxwnmiryHrNbWBfEVqHRQZ")
any_pubkey = Pubkey.from_string("BicJ4dmuWD3bfBrJyKKeqzczWDSGUepUpaKWmC6XRoJZ")

# Create the AccountMeta objects
auth_account_meta = AccountMeta(pubkey=auth_pubkey, is_signer=True, is_writable=True)
any_account_meta = AccountMeta(pubkey=any_pubkey, is_signer=False, is_writable=True)

# Create the Instruction object
instruction = Instruction(
    program_id=program_id,
    accounts=[auth_account_meta, any_account_meta],
    data= 0
)

TypeError: argument 'data': 'int' object cannot be converted to 'PyBytes'

In [38]:
from solders.message import Message
from solders.transaction import Transaction
from solders.hash import Hash
from solders.instruction import Instruction
from solders.pubkey import Pubkey

In [40]:
program_id = Pubkey.default()
arbitrary_instruction_data = bytes([1])
accounts = []
instruction = Instruction(program_id, arbitrary_instruction_data, accounts)
payer = Keypair()
message = Message([instruction], payer.pubkey())
blockhash = Hash.default()
tx = Transaction([payer], message, blockhash)

In [41]:
account

[]

In [33]:
solclient.get_account_info('8h3eQCUq4sfKraaSGgoQM2wt1kjCYq6BoaAkCrPt3waa')

TypeError: argument 'pubkey': 'str' object cannot be converted to 'Pubkey'

In [24]:
from web3 import Web3
# Load ABIs

w3 = Web3(Web3.HTTPProvider("https://solana-mainnet.g.alchemy.com/v2/KgpuMjqocuyieV47x-HcBYGyi8LrIFb8"))


with open('DlnSource.abi.json') as f:
    dln_source_abi = json.load(f)

with open('erc20.abi.json') as f:  # Load the ERC-20 ABI
    erc20_abi = json.load(f)

# Contract addresses
DLN_SOURCE_ADDRESS = "src5qyZHqTqecJV4aY6Cb6zDZLMDzrDKKezs22MPHr4"
SRC_CHAIN_TOKEN_IN_ADDRESS = "0xaf88d065e77c8cC2239327C5EDb3A432268e5831"  # USDC on Arbitrum

# Create contract instances
dln_source_contract = w3.eth.contract(address=DLN_SOURCE_ADDRESS, abi=dln_source_abi)

HTTPError: 400 Client Error: Bad Request for url: https://solana-mainnet.g.alchemy.com/v2/KgpuMjqocuyieV47x-HcBYGyi8LrIFb8

In [1]:
from solders.message import Message
from solders.keypair import Keypair
from solders.instruction import Instruction
from solders.hash import Hash
from solders.transaction import Transaction
from solders.pubkey import Pubkey



In [12]:
program_id = Pubkey.default()
arbitrary_instruction_data = bytes([1])
accounts = []
instruction = Instruction(program_id, arbitrary_instruction_data, accounts)
payer = Keypair()
message = Message([instruction], payer.pubkey())
blockhash = Hash.default()  # replace with a real blockhash
tx = Transaction([payer], message, blockhash)

TypeError: Account() takes no arguments

In [12]:
import json
from web3 import Web3


w3 = Web3(Web3.HTTPProvider('https://solana-mainnet.g.alchemy.com/v2/KgpuMjqocuyieV47x-HcBYGyi8LrIFb8'))

with open('DlnSource.abi.json') as f:
    dln_source_abi = json.load(f)

with open('erc20.abi.json') as f:  # Load the ERC-20 ABI
    erc20_abi = json.load(f)






In [15]:
w3.eth.Trans